In [ ]:
import pickle
import sys
import json
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import os
import cPickle
%matplotlib inline

In [ ]:
all_item_data = []
accumulator = ''
len1 = 0
len2 = 0
f = open('../../all_items.json')
f.read(1)
for subs in f.read():
    accumulator += subs
    if accumulator == ',':
        accumulator = ''
    try:
        new_record = json.loads(accumulator)
        all_item_data.append(new_record)
        print len(all_item_data)
        accumulator = ''
    except:
        pass

In [ ]:
for item in all_item_data:
    item['item_id'] = int(item['item_id'])

In [ ]:
all_item_set = set()
for item in all_item_data:
    all_item_set.add(item['item_id'])

In [119]:
len(all_item_set)

10258

In [118]:
cPickle.dump(all_item_data, open('processed_data/all_item_data.pkl','w'))

In [ ]:
user_item=json.load(open('../../australia_users_items.json'))

In [ ]:
user_item[0]

In [ ]:
bundle_data = json.load(open('../../bundle_data.json'))

In [ ]:
bundle_data[0]

In [103]:
bundle_item_map=[]
bundle_discount_map=[]
item_id_lookup=[]
id_item_lookup=dict()

bundle_id_lookup = []
id_bundle_lookup=dict()

items_id_set = set()
items_index_set = set()
bundle_id_set = set()
bundle_index_set = set()

index_bundle = 0
index_item = 0

def check_item_exist(index,index_item):
    
    if index in all_item_set:
        if index not in id_item_lookup:
            id_item_lookup[index]=index_item
            item_id_lookup.append(index)
            items_id_set.add(index)
            items_index_set.add(index_item)
            index_item = index_item + 1
        item_list.append(id_item_lookup[index])
    return item_list,index_item
    
    
for bundle in bundle_data:
    item_list = []
    bundle_discount_map.append(bundle['bundle_discount'])
    bundle_id_set.add(int(bundle['bundle_id']))
    bundle_index_set.add(index_bundle)
    bundle_id_lookup.append(int(bundle['bundle_id']))
    id_bundle_lookup[int(bundle['bundle_id'])] = index_bundle
    for item in bundle['items']:
        if item['item_id'] != '':
            if "," in item['item_id']:
                mul_item_list = item['item_id'].split(",")
                for id1 in mul_item_list:
                    sub_item_list, index_item = check_item_exist(int(id1),index_item)
                    item_list.append(sub_item_list)
            else:
                item_list,index_item = check_item_exist(int(item['item_id']),index_item)
    bundle_item_map.append(item_list)
    index_bundle = index_bundle+1        
    

In [107]:
user_bundle_map=dict()
user_item_map=dict()
user_id_lookup = []
id_user_lookup = dict()

index_user = 0
for user in user_item:
    item_list = []
    bundle_list = []
    for item in user['items']:
        item_index = int(item['item_id'])
        if item_index in items_set:
            item_list.append(id_item_lookup[item_index])
        elif item_index in bundle_set:
            bundle_list.append(id_bundle_lookup[item_index])
    if len(bundle_list) != 0:
        user_bundle_map[index_user] = bundle_list
        user_item_map[index_user] = item_list
        id_user_lookup[user['user_id']] = index_user
        user_id_lookup.append(user['user_id'])
        index_user = index_user + 1
        

In [113]:
cPickle.dump(user_id_lookup, open('processed_data/user_id_lookup.pkl','w'))
cPickle.dump(id_user_lookup, open('processed_data/id_user_lookup.pkl','w'))
cPickle.dump(user_bundle_map, open('processed_data/user_bundle_map.pkl','w'))
cPickle.dump(user_item_map, open('processed_data/user_item_map.pkl','w'))
cPickle.dump(bundle_item_map, open('processed_data/bundle_item_map.pkl','w'))
cPickle.dump(bundle_discount_map, open('processed_data/bundle_discount_map.pkl','w'))
cPickle.dump(item_id_lookup, open('processed_data/item_id_lookup.pkl','w'))
cPickle.dump(id_item_lookup, open('processed_data/id_item_lookup.pkl','w'))
cPickle.dump(bundle_id_lookup, open('processed_data/bundle_id_lookup.pkl','w'))
cPickle.dump(id_bundle_lookup, open('processed_data/id_bundle_lookup.pkl','w'))
cPickle.dump(items_id_set, open('processed_data/items_id_set.pkl','w'))
cPickle.dump(items_index_set, open('processed_data/items_index_set.pkl','w'))
cPickle.dump(bundle_id_set, open('processed_data/bundle_id_set.pkl','w'))
cPickle.dump(bundle_index_set, open('processed_data/bundle_id_set.pkl','w'))


In [109]:
user_item_played = np.zeros((len(user_item_map.keys()),len(items_set)))
for lines in user_item:
    user_id = lines['user_id']
    if user_id in id_user_lookup:
        user_index = id_user_lookup[user_id]
        for items in lines['items']:
            item_id = int(items['item_id'])
            if item_id in id_item_lookup:
                item_index = id_item_lookup[item_id]
                playtime = items['playtime_forever']
                user_item_played[user_index][item_index] = int(playtime)       

In [122]:
user_total_playtime = user_item_played.sum(axis=1)
user_item_played_percentage = np.zeros((len(user_item_map.keys()),len(items_set)),dtype=np.float)
for i in range(len(user_item_map.keys())):
    if user_total_playtime[i] != 0:
        for j in range(len(items_set)):
            user_item_played_percentage[i][j] = float(user_item_played[i][j]) / user_total_playtime[i]

In [123]:
cPickle.dump(user_item_played_percentage, open('processed_data/user_item_played_percentage.pkl','w'))

In [ ]:
item_data=pickle.load(open('processed_data/all_item_data.pkl','rb'))

In [ ]:
item_data_map=dict() #appid, item
tags_set=set()
for item in item_data:
    item_data_map[int(item['item_id'])]=item
    for tag in item['genre']:
        tags_set.add(tag)
tags_map=dict()
for i,tag in enumerate(tags_set):
    tags_map[tag]=i 
    
def get_feat(tags):
    feat=np.zeros(len(tags_map))
    for tag in tags:
        feat[tags_map[tag]]=1
    return feat
    
def get_item_data_map_id(index):
    id = item_id_lookup[index]
    for i in range(0,len(item_data)-1):
        if item_data[i]['item_id'] == id:
            return i
            
sgd_tags = np.array([get_feat(item_data[get_item_data_map_id(i)]['genre']) for i in items_set])
print(sgd_tags.shape)

In [ ]:
user_genre_played_percentage = np.dot(user_item_played_percentage,sgd_tags)
cPickle.dump(user_genre_played_percentage, open('processed_data/user_genre_played_percentage.pkl','w'))